# Feature Selection and LASSO - Algorithm to select optimal L1 penalty


* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In [1]:
import graphlab
import math
import random
import numpy
from matplotlib import pyplot as plt
%matplotlib inline

This non-commercial license of GraphLab Create for academic use is assigned to aguha2@jhu.edu and will expire on February 11, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1473048252.log


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms','sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.690934     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.739669     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [6]:
def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])
    
    #for i in range(deg+1):        
        #if w[i] !=0:
            #print i
            #print all_features[i-1]
    

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned polynomial for degree ' + str(deg) + ':'
    w.reverse()
    print numpy.poly1d(w)

In [7]:
def plot_data(data):    
    plt.plot(data,data['price'],'k.')
    plt.xlabel('x')
    plt.ylabel('y')

#plot_data(sales)

In [8]:
def plot_poly_predictions(data, model):
    plot_data(data)

    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1
    
    # Create 200 points in the x axis and compute the predicted value for each point
    #x_pred = graphlab.SFrame({'X1':[i/200.0 for i in range(200)]})
    y_pred = model.predict(polynomial_features(x_pred,deg))
    
    # plot predictions
    plt.plot(x_pred, y_pred, 'g-', label='degree ' + str(deg) + ' fit')
    plt.legend(loc='upper left')
    #plt.axis([0,1,-1.5,2])

Find what features had non-zero weight.

In [10]:
print 'number of nonzeros = %d' % (model_all.coefficients['value']).nnz()
print_coefficients(model_all)
print '\n'
# plt.figure()
# plot_poly_predictions(sales,model_all)
# plt.title('LASSO, lambda = %.2e, # nonzeros = %d' % (l1_penalty, (model_all.coefficients['value']).nnz()))

number of nonzeros = 6
Learned polynomial for degree 17:
       14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05




Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [11]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [12]:
def validation_sframe(dataset,features):
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    #poly_sframe['price'] = dataset['price']
    
    for i in range(0, len(features)): 
            name = str(features[i])
            poly_sframe[name] = dataset[name]
    return poly_sframe

In [13]:
validation_sframe(validation, all_features)

bedrooms,bedrooms_square,bathrooms,sqft_living,sqft_living_sqrt,sqft_lot,sqft_lot_sqrt,floors,floors_square
3.0,9.0,1.0,1180.0,34.3511280746,5650,75.1664818919,1.0,1.0
3.0,9.0,2.25,2570.0,50.6951674225,7242,85.0999412456,2.0,4.0
2.0,4.0,1.0,770.0,27.748873851,10000,100.0,1.0,1.0
3.0,9.0,2.0,1680.0,40.9878030638,8080,89.888820217,1.0,1.0
3.0,9.0,2.25,1715.0,41.4125584817,6819,82.5772365728,2.0,4.0
3.0,9.0,1.5,1060.0,32.5576411922,9711,98.5444062339,1.0,1.0
3.0,9.0,1.0,1780.0,42.1900462195,7470,86.4291617453,1.0,1.0
3.0,9.0,2.5,3560.0,59.6657355607,9796,98.9747442533,1.0,1.0
3.0,9.0,2.0,1890.0,43.4741302386,14040,118.490505949,2.0,4.0
3.0,9.0,1.0,1250.0,35.3553390593,9774,98.8635423197,1.0,1.0


Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

# Calculating best L2 penalty

In [14]:
import numpy as np
RSS_array=[]
RSS_min = 1e100
best_l1_penalty=10
for l1_penalty in np.logspace(1,7,num=13):
    #print l1_penalty
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, 
                                                  verbose = False)
    print l1_penalty
    print 'number of nonzeros = %d' % (model.coefficients['value']).nnz()
    print_coefficients(model)
    
    poly_validdata = validation_sframe(validation, all_features)
    poly_validdata['price'] = validation['price'] # add price to the data since it's the target

    predictions = model.predict(poly_validdata)
        # Then compute the residuals/errors
    rss_errors = poly_validdata['price'] - predictions
        # Then square and add them up
    rss_error_sq= rss_errors*rss_errors
        #RSS = RSS_ + str(degree)
    RSS=rss_error_sq.sum()
    print RSS
    RSS_array.append(RSS)
    if RSS < RSS_min:
        RSS_min=RSS
        best_l1_penalty=l1_penalty
    
    print '\n'
    


10.0
number of nonzeros = 18
Learned polynomial for degree 17:
       17         16         15         14        13        12
56.07 x  + 9.434 x  + 122.4 x  + 43.29 x  + 6207 x  + 6609 x 
              11             10             9            8         7
 + 9.331e+04 x  + 6.019e+05 x  + 1.292e+04 x + 2.12e+04 x + 148.3 x
             6        5         4             3       2
 + 0.003484 x + 1125 x + 39.12 x + 2.541e+04 x + 937 x + 7937 x + 1.899e+04
6.25766285142e+14


31.6227766017
number of nonzeros = 18
Learned polynomial for degree 17:
       17         16         15         14        13        12
56.07 x  + 9.434 x  + 122.4 x  + 43.29 x  + 6207 x  + 6609 x 
              11             10             9            8         7
 + 9.331e+04 x  + 6.019e+05 x  + 1.292e+04 x + 2.12e+04 x + 148.3 x
             6        5         4             3       2
 + 0.003484 x + 1125 x + 39.12 x + 2.541e+04 x + 937 x + 7937 x + 1.899e+04
6.25766285362e+14


100.0
number of nonzeros = 18
Learned

In [15]:
print RSS_min
print best_l1_penalty
print RSS_array

6.25766285142e+14
10.0
[625766285142459.9, 625766285362394.1, 625766286057885.0, 625766288257224.6, 625766295212186.8, 625766317206080.5, 625766386760658.1, 625766606749278.5, 625767302791634.1, 625769507643886.2, 625776517727024.0, 625799062845467.0, 625883719085425.2]


In [16]:
    poly_testdata = validation_sframe(testing, all_features)
    #poly_testdata
    poly_testdata['price'] = testing['price'] # add price to the data since it's the target
    
    model_bestl1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10, 
                                                  verbose = False)
    
    predictions_2 = model_bestl1.predict(poly_testdata)
        # Then compute the residuals/errors
    rss_errors = poly_testdata['price'] - predictions_2
        # Then square and add them up
    rss_error_sq= rss_errors*rss_errors
        #RSS = RSS_ + str(degree)
    RSS=rss_error_sq.sum()
    print RSS

1.56983602382e+14


## Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [25]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [26]:
l1_penalty_values = np.logspace(8, 10, num=20)
l1_penalty_values

array([  1.00000000e+08,   1.27427499e+08,   1.62377674e+08,
         2.06913808e+08,   2.63665090e+08,   3.35981829e+08,
         4.28133240e+08,   5.45559478e+08,   6.95192796e+08,
         8.85866790e+08,   1.12883789e+09,   1.43844989e+09,
         1.83298071e+09,   2.33572147e+09,   2.97635144e+09,
         3.79269019e+09,   4.83293024e+09,   6.15848211e+09,
         7.84759970e+09,   1.00000000e+10])

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [27]:
import numpy as np
RSS_min = 1e100
best_l1_penalty=10
for l1_penalty in np.logspace(8,10,num=20):
    #print l1_penalty
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, 
                                                  verbose = False)
    print l1_penalty
    print 'number of nonzeros = %d' % (model.coefficients['value']).nnz()
    print_coefficients(model)
    
    poly_validdata_2 = validation_sframe(validation, all_features)
    poly_validdata_2['price'] = validation['price'] # add price to the data since it's the target

    predictions = model.predict(poly_validdata_2)
        # Then compute the residuals/errors
    rss_errors = poly_validdata_2['price'] - predictions
        # Then square and add them up
    rss_error_sq= rss_errors*rss_errors
        #RSS = RSS_ + str(degree)
    RSS=rss_error_sq.sum()
    print RSS
    
    
    print '\n'
    


100000000.0
number of nonzeros = 18
Learned polynomial for degree 17:
       17        16         15         14        13        12
48.62 x  + 9.04 x  + 118.9 x  + 43.04 x  + 6139 x  + 6361 x 
              11             10             9            8       7
 + 9.107e+04 x  + 5.682e+05 x  + 1.247e+04 x + 2.07e+04 x + 144 x
            6        5         4             3         2
 - 0.02569 x + 1117 x + 39.04 x + 2.523e+04 x + 847.6 x + 7789 x + 2.509e+04
6.27492659875e+14


127427498.57
number of nonzeros = 18
Learned polynomial for degree 17:
       17         16       15         14        13        12
46.56 x  + 8.924 x  + 118 x  + 42.96 x  + 6118 x  + 6288 x 
              11             10             9             8         7
 + 9.044e+04 x  + 5.589e+05 x  + 1.234e+04 x + 2.055e+04 x + 142.5 x
            6        5         4             3         2
 - 0.01866 x + 1115 x + 39.01 x + 2.518e+04 x + 822.4 x + 7744 x + 2.675e+04
6.28210516771e+14


162377673.919
number of nonzeros = 

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [28]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [29]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
l1_penalty_values

array([  2.97635144e+09,   3.01931664e+09,   3.06228184e+09,
         3.10524703e+09,   3.14821223e+09,   3.19117743e+09,
         3.23414263e+09,   3.27710782e+09,   3.32007302e+09,
         3.36303822e+09,   3.40600341e+09,   3.44896861e+09,
         3.49193381e+09,   3.53489901e+09,   3.57786420e+09,
         3.62082940e+09,   3.66379460e+09,   3.70675980e+09,
         3.74972499e+09,   3.79269019e+09])

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [30]:
import numpy as np

for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    #print l1_penalty
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, 
                                                  verbose = False)
    print l1_penalty
    print 'number of nonzeros = %d' % (model.coefficients['value']).nnz()
    print_coefficients(model)
    
    poly_validdata_2 = validation_sframe(validation, all_features)
    poly_validdata_2['price'] = validation['price'] # add price to the data since it's the target

    predictions = model.predict(poly_validdata_2)
        # Then compute the residuals/errors
    rss_errors = poly_validdata_2['price'] - predictions
        # Then square and add them up
    rss_error_sq= rss_errors*rss_errors
        #RSS = RSS_ + str(degree)
    RSS=rss_error_sq.sum()
    print RSS
    
    
    print '\n'
    


2976351441.63
number of nonzeros = 10
Learned polynomial for degree 17:
      15         14        13             11        8         5
12.8 x  + 32.74 x  + 3579 x  + 1.133e+04 x  + 3666 x + 789.3 x
          4             3
 + 34.14 x + 1.796e+04 x + 2182 x + 1.961e+05
9.66925692362e+14


3019316638.95
number of nonzeros = 10
Learned polynomial for degree 17:
       15         14        13        11        8         5         4
11.05 x  + 32.54 x  + 3528 x  + 9877 x  + 3358 x + 782.2 x + 34.02 x
              3
 + 1.781e+04 x + 2067 x + 1.986e+05
9.74019450085e+14


3062281836.27
number of nonzeros = 10
Learned polynomial for degree 17:
       15         14        13        11        8       5        4
9.306 x  + 32.33 x  + 3478 x  + 8419 x  + 3050 x + 775 x + 33.9 x
              3
 + 1.766e+04 x + 1952 x + 2.01e+05
9.81188367942e+14


3105247033.59
number of nonzeros = 10
Learned polynomial for degree 17:
       15         14        13        11        8       5         4
7.513 x  +

Value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
Features in this model have non-zero coefficients?

In [ ]:
3448968612.16


In [ ]:
       14        13         5         4             3
30.01 x  + 2899 x  + 690.1 x + 32.41 x + 1.587e+04 x + 661.7 x + 2.223e+05

3
bathrooms
4
sqft_living
5
sqft_living_sqrt
13
grade
14
sqft_above
